# Test Download Performance with Streaming

Questo notebook testa le funzionalità core del metodo `export` con streaming per migliorare le performance del download di dataset dall'API OpenData.


In [4]:
import asyncio
import time
import httpx
from pathlib import Path
import sys

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent))

from backend.opendata_api.client import BolognaOpenData


## Versione Non-Streaming con HTTP/2

Test della versione che usa `r.content` con HTTP/2 abilitato.


In [5]:
async def export_http2(dataset_id: str, fmt: str = "parquet") -> bytes:
    """Versione con HTTP/2: carica tutto in memoria con r.content"""
    # Create client with HTTP/2 enabled
    client_http2 = httpx.AsyncClient(
        base_url="https://opendata.comune.bologna.it/api/explore/v2.1",
        timeout=httpx.Timeout(connect=10.0, read=600.0, write=10.0, pool=5.0),
        limits=httpx.Limits(max_keepalive_connections=10, max_connections=20, keepalive_expiry=60.0),
        http2=True  # Enable HTTP/2
    )
    try:
        r = await client_http2.get(
            f"/catalog/datasets/{dataset_id}/exports/{fmt}"
        )
        r.raise_for_status()
        return r.content
    finally:
        await client_http2.aclose()

# Test
dataset_id = "rilevazione-flusso-veicoli-tramite-spire-anno-2025"

print(f"Testing HTTP/2 download of: {dataset_id}")
start = time.time()
data_http2 = await export_http2(dataset_id, "parquet")
elapsed_http2 = time.time() - start
size_mb = len(data_http2) / (1024 * 1024)

print(f"✅ HTTP/2 download completed in {elapsed_http2:.2f}s")
print(f"📦 Size: {size_mb:.2f} MB")
print(f"⚡ Speed: {size_mb/elapsed_http2:.2f} MB/s")


Testing HTTP/2 download of: rilevazione-flusso-veicoli-tramite-spire-anno-2025
✅ HTTP/2 download completed in 99.36s
📦 Size: 13.94 MB
⚡ Speed: 0.14 MB/s


## Versione con Streaming + Bytearray

Versione ottimizzata che usa streaming con bytearray e chunk size di 2MB (implementata nel client).


In [6]:
async def export_streaming_bytearray(
    client: BolognaOpenData, 
    dataset_id: str, 
    fmt: str = "parquet"
) -> bytes:
    """Versione esplicita con streaming + bytearray: mostra chiaramente lo streaming"""
    await client._ensure_client_ready()
    try:
        # Use streaming with explicit async context manager
        async with client._client.stream(
            "GET",
            f"/catalog/datasets/{dataset_id}/exports/{fmt}"
        ) as response:
            response.raise_for_status()
            
            # Use bytearray for efficient accumulation (faster than list + join)
            buffer = bytearray()
            async for chunk in response.aiter_bytes(chunk_size=512):  # 2MB chunks
                buffer.extend(chunk)
            
            return bytes(buffer)
            
    except (RuntimeError, ConnectionError, httpx.ReadError) as e:
        if "closed" in str(e).lower() or isinstance(e, (ConnectionError, httpx.ReadError)):
            # Retry once on connection errors
            await client._ensure_client_ready()
            async with client._client.stream(
                "GET",
                f"/catalog/datasets/{dataset_id}/exports/{fmt}"
            ) as response:
                response.raise_for_status()
                buffer = bytearray()
                async for chunk in response.aiter_bytes(chunk_size=2 * 1024 * 1024):
                    buffer.extend(chunk)
                return bytes(buffer)
        raise

# Test
client = BolognaOpenData()

print(f"Testing streaming + bytearray download of: {dataset_id}")
start = time.time()
data_streaming = await export_streaming_bytearray(client, dataset_id, "parquet")
elapsed_streaming = time.time() - start
size_mb_streaming = len(data_streaming) / (1024 * 1024)

print(f"\n✅ Streaming + bytearray download completed in {elapsed_streaming:.2f}s")
print(f"📦 Size: {size_mb_streaming:.2f} MB")
print(f"⚡ Speed: {size_mb_streaming/elapsed_streaming:.2f} MB/s")


Testing streaming + bytearray download of: rilevazione-flusso-veicoli-tramite-spire-anno-2025

✅ Streaming + bytearray download completed in 99.40s
📦 Size: 13.94 MB
⚡ Speed: 0.14 MB/s


In [7]:
# Verifica che i dati siano identici
assert data_http2 == data_streaming, "❌ Data mismatch!"
print("✅ Data integrity check passed: both methods return identical data\n")

# Confronto
print("📊 Performance Comparison:")
print(f"{'Method':<30} {'Time (s)':<12} {'Speed (MB/s)':<15} {'vs HTTP/2':<15}")
print("-" * 72)
print(f"{'HTTP/2 (r.content)':<30} {elapsed_http2:<12.2f} {size_mb/elapsed_http2:<15.2f} {'baseline':<15}")
print(f"{'Streaming + bytearray':<30} {elapsed_streaming:<12.2f} {size_mb_streaming/elapsed_streaming:<15.2f} {f'{((elapsed_http2 - elapsed_streaming) / elapsed_http2)*100:+.1f}%':<15}")

if elapsed_streaming < elapsed_http2:
    improvement = ((elapsed_http2 - elapsed_streaming) / elapsed_http2) * 100
    print(f"\n🚀 Streaming + bytearray is {improvement:.1f}% faster than HTTP/2!")
else:
    slowdown = ((elapsed_streaming - elapsed_http2) / elapsed_http2) * 100
    print(f"\n⚠️  Streaming + bytearray is {slowdown:.1f}% slower than HTTP/2")

# Cleanup
await client.close()
print("\n✅ Client closed")


✅ Data integrity check passed: both methods return identical data

📊 Performance Comparison:
Method                         Time (s)     Speed (MB/s)    vs HTTP/2      
------------------------------------------------------------------------
HTTP/2 (r.content)             99.36        0.14            baseline       
Streaming + bytearray          99.40        0.14            -0.0%          

⚠️  Streaming + bytearray is 0.0% slower than HTTP/2

✅ Client closed


In [10]:
import os
import boto3
from botocore.client import Config
from dotenv import load_dotenv

# Check S3 configuration
load_dotenv()
s3_bucket = os.getenv("S3_BUCKET")
if not s3_bucket:
    print("⚠️  S3_BUCKET not set. Skipping S3 upload test.")
    print("   Set S3_BUCKET, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY to test upload.")
else:
    # 1. Download con HTTP/2
    print(f"📥 Step 1: Downloading {dataset_id} with HTTP/2...")
    download_start = time.time()
    data_downloaded = await export_http2(dataset_id, "parquet")
    download_time = time.time() - download_start
    size_mb = len(data_downloaded) / (1024 * 1024)
    
    print(f"   ✅ Download completed in {download_time:.2f}s")
    print(f"   📦 Size: {size_mb:.2f} MB")
    print(f"   ⚡ Download speed: {size_mb/download_time:.2f} MB/s\n")
    
    # 2. Upload su S3
    print(f"📤 Step 2: Uploading to S3 (input/datasets/{dataset_id}.parquet)...")
    upload_start = time.time()
    
    # Setup S3 client
    region = os.getenv("AWS_REGION", "eu-central-1")
    s3_client = boto3.client(
        "s3",
        region_name=region,
        config=Config(signature_version='s3v4')
    )
    
    # S3 key: input/datasets/{dataset_id}.parquet (coerente con load_dataset_tool)
    s3_key = f"input/datasets/{dataset_id}.parquet"
    
    # Upload to S3
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=s3_key,
        Body=data_downloaded,
        ContentType="application/parquet"
    )
    
    upload_time = time.time() - upload_start
    
    print(f"   ✅ Upload completed in {upload_time:.2f}s")
    print(f"   ⚡ Upload speed: {size_mb/upload_time:.2f} MB/s")
    print(f"   🔗 S3 location: s3://{s3_bucket}/{s3_key}\n")
    
    # 3. Summary
    total_time = download_time + upload_time
    print("📊 Summary:")
    print(f"   {'Operation':<20} {'Time (s)':<15} {'Speed (MB/s)':<15}")
    print(f"   {'-'*20} {'-'*15} {'-'*15}")
    print(f"   {'Download (HTTP/2)':<20} {download_time:<15.2f} {size_mb/download_time:<15.2f}")
    print(f"   {'Upload (S3)':<20} {upload_time:<15.2f} {size_mb/upload_time:<15.2f}")
    print(f"   {'TOTAL':<20} {total_time:<15.2f} {'':<15}")
    print(f"\n   📈 Upload overhead: {upload_time:.2f}s ({upload_time/total_time*100:.1f}% of total time)")
    
    # Verify upload
    try:
        s3_client.head_object(Bucket=s3_bucket, Key=s3_key)
        print(f"   ✅ Verified: file exists in S3")
    except Exception as e:
        print(f"   ⚠️  Warning: could not verify S3 upload: {e}")

📥 Step 1: Downloading rilevazione-flusso-veicoli-tramite-spire-anno-2025 with HTTP/2...
   ✅ Download completed in 104.70s
   📦 Size: 13.94 MB
   ⚡ Download speed: 0.13 MB/s

📤 Step 2: Uploading to S3 (input/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2025.parquet)...
   ✅ Upload completed in 1.73s
   ⚡ Upload speed: 8.05 MB/s
   🔗 S3 location: s3://lg-urban-prod/input/datasets/rilevazione-flusso-veicoli-tramite-spire-anno-2025.parquet

📊 Summary:
   Operation            Time (s)        Speed (MB/s)   
   -------------------- --------------- ---------------
   Download (HTTP/2)    104.70          0.13           
   Upload (S3)          1.73            8.05           
   TOTAL                106.43                         

   📈 Upload overhead: 1.73s (1.6% of total time)
   ✅ Verified: file exists in S3
